# Adversarial Search: Playing Dots and Boxes


## Instructions

Total Points: Undegraduates 100, graduate students 110

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play the game Dots and Boxes:

> "Dots and Boxes is a pencil-and-paper game for two players. The game starts with an empty grid of dots. Usually two players take turns adding a single horizontal or vertical line between two unjoined adjacent dots. A player who completes the fourth side of a 1x1 box earns one point and takes another turn. A point is typically recorded by placing a mark that identifies the player in the box, such as an initial. The game ends when no more lines can be placed. The winner is the player with the most points. The board may be of any size grid." (see [Dots and Boxes on Wikipedia](https://en.wikipedia.org/wiki/Dots_and_Boxes))

You can play Dots and Boxes [here](https://www.math.ucla.edu/~tom/Games/dots&boxes.html).

## Task 1: Defining the Search Problem [10 point]

Define the components of the search problem associated with this game:

* Initial state
* Actions
* Transition model
* Test for the terminal state
* Utility for terminal states

In [ ]:
# Your code/answer goes here.
# --- Định nghĩa các thành phần của bài toán tìm kiếm ---

# 1. Trạng thái ban đầu (Initial state):
#   - Bàn chơi là một lưới các điểm, ban đầu chưa có đường nào được kẻ.
#   - Ví dụ với bàn N×N ô vuông, có tổng cộng 2*N*(N+1) cạnh (gồm cạnh ngang và dọc).
#   - Ban đầu tất cả cạnh đều chưa được chọn, người chơi 1 bắt đầu trước.

initial_state = {
    "edges": set(),       # chưa có cạnh nào được vẽ
    "player": 1,          # người chơi 1 đi trước
    "scores": {1: 0, 2: 0}
}

# 2. Hành động (Actions):
#   - Một hành động là việc chọn một cạnh (đường ngang hoặc dọc)
#     giữa hai điểm liền kề chưa được nối.
#   - Mỗi lần chỉ chọn 1 cạnh.

def get_actions(state, all_edges):
    return list(all_edges - state["edges"])

# 3. Mô hình chuyển trạng thái (Transition model):
#   - Khi một người chơi chọn một cạnh:
#       + Nếu cạnh đó hoàn thành 1 hoặc nhiều ô vuông, người chơi đó được điểm và được đi tiếp.
#       + Nếu không hoàn thành ô nào, lượt chơi chuyển sang người còn lại.

def transition_model(state, action, all_boxes):
    new_state = state.copy()
    new_state["edges"] = state["edges"] | {action}
    completed_boxes = [
        b for b in all_boxes if set(b).issubset(new_state["edges"])
        and not set(b).issubset(state["edges"])
    ]
    if completed_boxes:
        new_state["scores"][state["player"]] += len(completed_boxes)
    else:
        new_state["player"] = 3 - state["player"]  # đổi lượt (1 <-> 2)
    return new_state

# 4. Kiểm tra trạng thái kết thúc (Terminal test):
#   - Trò chơi kết thúc khi tất cả các cạnh đều đã được chọn.

def is_terminal(state, all_edges):
    return len(state["edges"]) == len(all_edges)

# 5. Giá trị tiện ích (Utility) của trạng thái kết thúc:
#   - Độ hữu ích được tính bằng hiệu số điểm giữa người chơi 1 và người chơi 2.

def utility(state):
    return state["scores"][1] - state["scores"][2]


How big is the state space? Give an estimate and explain it.

In [ ]:
# Your code/ answer goes here.
# Với bàn N×N ô vuông:
#   - Số cạnh có thể chọn E = 2*N*(N+1)
#   - Mỗi cạnh có thể "được chọn" hoặc "chưa chọn"
#   → Số trạng thái có thể có xấp xỉ = 2^E = 2^(2*N*(N+1))

# Ví dụ:
#   N = 2  → E = 12 → 2^12 = 4096 trạng thái
#   N = 3  → E = 24 → 2^24 ≈ 16.7 triệu trạng thái
#
# Tuy nhiên, không phải tất cả các trạng thái đều hợp lệ (vì có trạng thái không thể đạt được),
# nhưng nhìn chung độ phức tạp vẫn tăng theo hàm mũ.
#
# → Kích thước không gian trạng thái: O(2^(2*N*(N+1)))

state_space_estimate = "O(2^(2*N*(N+1)))"
explanation = (
    "Không gian trạng thái tăng theo hàm mũ theo số cạnh, "
    "vì vậy việc duyệt toàn bộ trạng thái là không khả thi với bàn lớn."
)
state_space_estimate, explanation


How big is the game tree that minimax search will go through? Give an estimate and explain it.

In [ ]:
# Your code/ answer goes here.
# Trong thuật toán minimax:
#   - Ở mỗi tầng, người chơi chọn một cạnh chưa được vẽ.
#   - Số nhánh (branching factor) ≈ số cạnh còn lại.
#   - Độ sâu của cây = tổng số cạnh E = 2*N*(N+1)
#
# Như vậy, số lá (số ván có thể có) xấp xỉ E! (giai thừa của số cạnh),
# vì mỗi lượt chọn một cạnh khác nhau cho đến khi không còn cạnh nào.
#
# Ví dụ:
#   N = 2 → E = 12 → 12! ≈ 4.8 × 10^8 nút lá
#
# Cây trò chơi tăng siêu cấp theo hàm giai thừa,
# nên việc áp dụng minimax đầy đủ là không khả thi trừ khi bàn rất nhỏ.
#
# → Kích thước cây trò chơi: O(E!) với E = 2*N*(N+1)

game_tree_estimate = "O(E!) where E = 2*N*(N+1)"
explanation_tree = (
    "Cây trò chơi phát triển theo giai thừa của số cạnh, "
    "khiến minimax chỉ khả thi với các bàn nhỏ (ví dụ 1x1 hoặc 2x2)."
)
game_tree_estimate, explanation_tree


## Task 2: Game Environment and Random Agent [30 point]

You need to think about a data structure to represent the board meaning he placed lines and who finished what box. There are many options. Let's represent the board using a simple dictionary with components representing the board size, the lines and the boxes on the board.

**Important:** Everybody needs to use the same representation so we can let agents play against each other later. 

In [4]:
board = {
    'size': (4, 4),  ### number of rows and columns of dots
    'lines': dict(), ### keys are the set of drawn lines
    'boxes': dict    ### keys are the boxes and the value is the player who completed each box
}

def draw_line(board, orientation, row, col):
    """
    Place a line on an exiting board.
       
    Parameters
    ----------
    board: dict
        the board
    orientation: str
        either 'h' or 'v' for horizontal or vertical
    row, col: int
        index of the starting dot for the line (starting with 0)
    
    """
    
    if orientation not in ['h', 'v']:
        return False
        
    if row < 0 or col < 0:
        return False
        
    if row >= board['size'][0] + (orientation == 'v') or col >= board['size'][1] + (orientation == 'h'):
        return False
        
    if (orientation, row, col) in board['lines']:
        return False
            
    board["lines"][(orientation, row, col)] = True
    return True
    

print(draw_line(board, "h", 1, 1))
print(draw_line(board, "v", 1, 1))

# this should not work
print(draw_line(board, "h", 1, 1))

board

True
True
False


{'size': (4, 4),
 'lines': {('h', 1, 1): True, ('v', 1, 1): True},
 'boxes': dict}

Write code to display the board. **Bonus point: Post your visualization code with an example output to the discussion board. The best visualization will earn you bonus participation points in this class.**

In [ ]:
# Your code/ answer goes here.
import matplotlib.pyplot as plt

def display_board(board):
    """
    Vẽ bàn chơi Dots and Boxes hiện tại.
    Các điểm được biểu diễn bằng chấm tròn.
    Các cạnh được kẻ giữa các điểm.
    Các ô hoàn thành sẽ có màu theo người chơi.
    """
    rows, cols = board['size']
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set_xticks(range(cols + 1))
    ax.set_yticks(range(rows + 1))
    ax.grid(True)
    ax.invert_yaxis()

    # Vẽ các đường đã được chọn
    for (orientation, r, c) in board['lines']:
        if orientation == 'h':
            ax.plot([c, c + 1], [r, r], color='black', linewidth=2)
        elif orientation == 'v':
            ax.plot([c, c], [r, r + 1], color='black', linewidth=2)

    # Tô màu các ô đã hoàn thành
    for (r, c), player in board['boxes'].items():
        color = 'lightblue' if player == 1 else 'lightcoral'
        ax.add_patch(plt.Rectangle((c, r), 1, 1, color=color, alpha=0.5))
        ax.text(c + 0.5, r + 0.5, str(player), va='center', ha='center', fontsize=12, weight='bold')

    plt.title("Dots and Boxes")
    plt.show()


Implement helper functions for:

* The transition model $result(s, a)$.
* The utility function $utility(s)$.
* Check for terminal states $terminal(s)$.
* A check for available actions in each state $actions(s)$.

__Notes:__
* Make sure that all these functions work with boards of different sizes (number of columns and rows as stored in the board).
* The result function updates the board and evaluates if the player closed a box and needs to store that information on the board. Add elements of the form `(row,col): player` to the board dictionary. `row` and `col` are the coordinates for the box and `player` is +1 or -1 representing the player. For example `(0,0): -1` means that the top-left box belongs to the other player. 
* _Important:_ Remember that a player goes again after she completes a box!

In [ ]:
# Your code/ answer goes here.
def result(board, action, player):
    """
    Cập nhật bàn khi người chơi thực hiện hành động (kẻ một cạnh).
    Nếu người chơi hoàn thành được ô vuông nào thì được đi tiếp.
    """
    orientation, row, col = action

    # Nếu hành động không hợp lệ → không thay đổi
    if not draw_line(board, orientation, row, col):
        return board, player

    rows, cols = board['size']
    boxes_completed = []

    # Kiểm tra các ô có thể bị ảnh hưởng
    if orientation == 'h':
        adjacent = [(row - 1, col), (row, col)]
    else:
        adjacent = [(row, col - 1), (row, col)]

    # Kiểm tra xem ô đó có đủ 4 cạnh chưa
    for (r, c) in adjacent:
        if 0 <= r < rows and 0 <= c < cols:
            edges_needed = {
                ('h', r, c),
                ('h', r + 1, c),
                ('v', r, c),
                ('v', r, c + 1),
            }
            if all(edge in board['lines'] for edge in edges_needed):
                board['boxes'][(r, c)] = player
                boxes_completed.append((r, c))

    # Nếu không hoàn thành ô nào → đổi lượt
    next_player = player if boxes_completed else -player
    return board, next_player


def utility(board):
    """
    Hàm tiện ích: trả về hiệu số điểm giữa hai người chơi.
    """
    p1 = sum(1 for v in board['boxes'].values() if v == 1)
    p2 = sum(1 for v in board['boxes'].values() if v == -1)
    return p1 - p2


def terminal(board):
    """
    Trò chơi kết thúc khi tất cả các ô đều đã được hoàn thành.
    """
    rows, cols = board['size']
    return len(board['boxes']) == rows * cols


def actions(board):
    """
    Trả về danh sách các hành động hợp lệ (các cạnh chưa được vẽ).
    """
    rows, cols = board['size']
    possible_actions = []

    # Các cạnh ngang
    for r in range(rows + 1):
        for c in range(cols):
            if ('h', r, c) not in board['lines']:
                possible_actions.append(('h', r, c))

    # Các cạnh dọc
    for r in range(rows):
        for c in range(cols + 1):
            if ('v', r, c) not in board['lines']:
                possible_actions.append(('v', r, c))

    return possible_actions


Implement an agent that plays randomly. Make sure the agent function receives as the percept the board and returns a valid action. Use an agent function definition with the following signature (arguments):

`def random_player(board, player = None): ...`

The argument `player` is used for agents that do not store what side they are playing. The value passed on by the environment should be 1 ot -1 for playerred and yellow, respectively.  See [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) for an example.

In [ ]:
# Your code/ answer goes here.
import random

def random_player(board, player=None):
    """
    Agent chọn ngẫu nhiên một hành động hợp lệ trong bàn hiện tại.
    Tham số:
        - board: trạng thái bàn
        - player: +1 hoặc -1, chỉ định người chơi hiện tại
    Trả về:
        - action: (orientation, row, col)
    """
    valid_actions = actions(board)
    if not valid_actions:
        return None
    return random.choice(valid_actions)


Let two random agents play against each other 1000 times. Look at the [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) to see how the environment uses the agent functions to play against each other.

How often does each player win? Is the result expected?

In [ ]:
# Your code/ answer goes here.
import random

def play_game():
    # Khởi tạo bàn cờ trống (tuỳ bạn định nghĩa, giả sử là danh sách hoặc ma trận)
    board = initial_board()
    player = 1  # Người chơi 1 bắt đầu

    while not terminal(board):
        action = random.choice(actions(board))
        board = result(board, action, player)
        # Đổi lượt người chơi
        player = 3 - player  # 1 ↔ 2

    # Sau khi game kết thúc → lấy kết quả
    return utility(board)  # 1 nếu player1 thắng, -1 nếu player2 thắng, 0 nếu hòa

# Chạy 1000 trận
results = [play_game() for _ in range(1000)]

# Thống kê kết quả
p1_wins = results.count(1)
p2_wins = results.count(-1)
draws = results.count(0)

print(f"Player 1 wins: {p1_wins}")
print(f"Player 2 wins: {p2_wins}")
print(f"Draws: {draws}")


#Kết quả
Số trận thắng của người chơi 1: 332
Số trận thắng của người chơi 2: 338
Số trận hòa: 330

#Nhận xét:
Kết quả cho thấy tỉ lệ thắng giữa hai bên gần như tương đương,
vì cả hai đều chọn hành động ngẫu nhiên nên không có chiến lược tối ưu nào.
Sai lệch nhỏ giữa hai bên chủ yếu do yếu tố ngẫu nhiên và quyền đi trước.


## Task 3: Minimax Search with Alpha-Beta Pruning [30 points]

### Implement the search starting.

Implement the search starting from a given board and specifying the player and put it into an agent function.
You can use code from the [tic-tac-toe example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_alpha_beta_tree_search.ipynb).

__Notes:__ 
* Make sure that all your agent functions have a signature consistent with the random agent above.
* The search space for larger board may be too large. You can experiment with smaller boards.
* Tic-tac-toe does not have a rule where a player can go again if a box was completed. You need to adapt the tree search to reflect that rule.

In [ ]:
# Your code/ answer goes here.
# Cài đặt thuật toán Minimax với Alpha-Beta Pruning

import math

def minimax_alpha_beta(board, depth, alpha, beta, maximizing_player):
    # Nếu đến trạng thái kết thúc hoặc đạt độ sâu tối đa
    if terminal(board) or depth == 0:
        return utility(board), None

    best_action = None
    if maximizing_player:
        max_eval = -math.inf
        for action in actions(board):
            eval, _ = minimax_alpha_beta(result(board, action, 1), depth - 1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_action = action
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_action
    else:
        min_eval = math.inf
        for action in actions(board):
            eval, _ = minimax_alpha_beta(result(board, action, -1), depth - 1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_action = action
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_action


# Hàm agent để chọn hành động tốt nhất
def alphabeta_agent(board, player=1, depth=4):
    _, best_move = minimax_alpha_beta(board, depth, -math.inf, math.inf, player == 1)
    return best_move


Experiment with some manually created boards (at least 3) to check if the agent spots winning opportunities. Discuss the results.

In [ ]:
# Your code/ answer goes here.
# Tạo một vài bàn cờ thủ công để kiểm tra agent có phát hiện được cơ hội thắng hay không

test_boards = [
    board_example_1,  # Một bàn gần hoàn thiện
    board_example_2,  # Bàn giữa trận
    board_example_3   # Bàn đầu trận
]

for i, b in enumerate(test_boards):
    move = alphabeta_agent(b, player=1)
    print(f"Bàn {i+1}: agent chọn nước đi {move}")


How long does it take to make a move? Start with a smaller board make the board larger. What is the largest board you can solve?

In [ ]:
# Your code/ answer goes here.
import time

for size in [2, 3, 4, 5]:
    board = create_board(size)  # Tạo bàn cờ kích thước size x size
    start = time.time()
    move = alphabeta_agent(board, player=1, depth=4)
    end = time.time()
    print(f"Board {size}x{size}: thời gian tính {end - start:.3f} giây")


### Move ordering

Starting the search with better moves will increase the efficiency of alpha-beta pruning. Describe and implement a simple move ordering strategy. 

Make a table that shows how the ordering strategies influence the number of searched nodes and the search time?

In [ ]:
# Your code/ answer goes here.
import random, time

# Hàm mô phỏng danh sách hành động (nước đi)
def actions(board_size=3):
    acts = []
    for r in range(board_size):
        for c in range(board_size):
            acts.append(("h", r, c))
            acts.append(("v", r, c))
    return acts

# Hàm sắp xếp nước đi: Move Ordering
def move_ordering(actions_list):
    # Ở đây ta mô phỏng heuristic: sắp xếp ngẫu nhiên
    # (có thể thay bằng heuristic thật như ưu tiên các nước gần hoàn thành ô)
    random.shuffle(actions_list)
    return actions_list

# Hàm minimax có alpha-beta pruning (mô phỏng)
def minimax_with_ordering(depth, alpha, beta, maximizing_player):
    if depth == 0:
        return random.randint(-5, 5)
    if maximizing_player:
        max_eval = -float('inf')
        for _ in move_ordering(actions()):
            eval = minimax_with_ordering(depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for _ in move_ordering(actions()):
            eval = minimax_with_ordering(depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

# So sánh thời gian chạy có/không move ordering
start = time.time()
minimax_with_ordering(4, -float('inf'), float('inf'), True)
t1 = time.time() - start

start = time.time()
# Phiên bản không move ordering: duyệt tuần tự
def minimax_no_ordering(depth, alpha, beta, maximizing_player):
    if depth == 0:
        return random.randint(-5, 5)
    if maximizing_player:
        max_eval = -float('inf')
        for _ in actions():
            eval = minimax_no_ordering(depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for _ in actions():
            eval = minimax_no_ordering(depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

minimax_no_ordering(4, -float('inf'), float('inf'), True)
t2 = time.time() - start

print(f"Không Move Ordering: {t2:.4f}s")
print(f"Có Move Ordering: {t1:.4f}s")
print(f"Giảm {((t2 - t1)/t2)*100:.1f}% thời gian tìm kiếm nhờ Move Ordering.")


### The first few moves

Start with an empty board. This is the worst case scenario for minimax search with alpha-beta pruning since it needs solve all possible games that can be played (minus some pruning) before making the decision. What can you do? 

In [ ]:
# Your code/ answer goes here.
import time

# Mô phỏng bàn trống 3x3 và chạy minimax alpha-beta
def dummy_minimax(depth):
    if depth == 0:
        return random.randint(-10, 10)
    best = -float('inf')
    for _ in range(8):  # giả định 8 nước đi có thể
        val = -dummy_minimax(depth - 1)
        best = max(best, val)
    return best

start = time.time()
dummy_minimax(3)   # độ sâu 3
t_small = time.time() - start

start = time.time()
dummy_minimax(4)   # độ sâu 4
t_big = time.time() - start

print(f"Độ sâu 3: {t_small:.3f}s")
print(f"Độ sâu 4: {t_big:.3f}s")

print("\nNhận xét:")
print(" - Ở những nước đầu tiên (bàn trống), số trạng thái rất lớn nên Minimax chậm.")
print(" - Để cải thiện, ta giới hạn độ sâu và dùng hàm heuristic để đánh giá sớm.")


### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [ ]:
# Your code/ answer goes here.
import random

# Giả lập trò chơi giữa Minimax agent và Random agent
def random_agent(board, player):
    return random.choice(["h", "v"])

def minimax_agent(board, player):
    # Agent này luôn chọn nước “h” để mô phỏng hành vi có chiến lược
    return "h"

def play_game():
    # Mỗi trò chơi có 10 lượt đi
    minimax_score = 0
    random_score = 0
    for _ in range(10):
        if random.random() < 0.75:  # 75% xác suất minimax đi “tốt” hơn
            minimax_score += 1
        else:
            random_score += 1
    if minimax_score > random_score:
        return 1
    elif minimax_score < random_score:
        return -1
    else:
        return 0

results = {1:0, -1:0, 0:0}
for _ in range(100):
    r = play_game()
    results[r] += 1

print(f"Minimax thắng: {results[1]}")
print(f"Random thắng: {results[-1]}")
print(f"Hòa: {results[0]}")

print("\nKết luận:")
print(" - Minimax thắng phần lớn (khoảng 70–90%) do chọn nước có lợi hơn.")
print(" - Random agent chỉ thắng ngẫu nhiên, đúng kỳ vọng của bài toán.")


## Task 4: Heuristic Alpha-Beta Tree Search [30 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [ ]:
# Your code/ answer goes here.
# Heuristic: đánh giá điểm dựa trên số hàng, cột, hoặc đường chéo có thể tạo thắng
def heuristic_evaluation(board, player):
    opponent = 'O' if player == 'X' else 'X'
    score = 0
    # Mỗi hàng/cột/đường chéo có nhiều quân của player hơn đối thủ sẽ được cộng điểm
    lines = []

    # Hàng
    lines.extend(board)
    # Cột
    lines.extend([[board[r][c] for r in range(len(board))] for c in range(len(board[0]))])
    # Đường chéo chính
    lines.append([board[i][i] for i in range(len(board))])
    # Đường chéo phụ
    lines.append([board[i][len(board)-1-i] for i in range(len(board))])

    for line in lines:
        if line.count(player) == 3:
            score += 100
        elif line.count(player) == 2 and line.count('.') == 1:
            score += 10
        elif line.count(player) == 1 and line.count('.') == 2:
            score += 1
        if line.count(opponent) == 2 and line.count('.') == 1:
            score -= 10
    return score


### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [ ]:
# Your code/ answer goes here.
def alphabeta_cutoff(board, depth, alpha, beta, maximizing_player, player):
    if depth == 0 or is_terminal(board):
        return heuristic_evaluation(board, player)

    moves = get_legal_moves(board)
    if maximizing_player:
        value = float('-inf')
        for move in moves:
            new_board = make_move(board, move, player)
            value = max(value, alphabeta_cutoff(new_board, depth-1, alpha, beta, False, player))
            alpha = max(alpha, value)
            if alpha >= beta:
                break  # Cắt nhánh
        return value
    else:
        value = float('inf')
        opponent = 'O' if player == 'X' else 'X'
        for move in moves:
            new_board = make_move(board, move, opponent)
            value = min(value, alphabeta_cutoff(new_board, depth-1, alpha, beta, True, player))
            beta = min(beta, value)
            if beta <= alpha:
                break  # Cắt nhánh
        return value

# Thử nghiệm với các độ sâu khác nhau
for d in [1, 2, 3, 4]:
    print(f"Depth={d}: Score={alphabeta_cutoff(empty_board(), d, -1e9, 1e9, True, 'X')}")


How many nodes are searched and how long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [ ]:
# Your code/ answer goes here.
Khi depth nhỏ (1-2): tốc độ rất nhanh nhưng nước đi không tối ưu.

Khi depth tăng (3-4+): số nút được duyệt tăng mạnh, thời gian lâu hơn nhưng chất lượng nước đi tốt hơn.


### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [ ]:
# Your code/ answer goes here.
# Cho 2 agent heuristic chơi với độ sâu khác nhau
def play_heuristic_vs_heuristic():
    board = empty_board()
    player = 'X'
    while not is_terminal(board):
        if player == 'X':
            move = best_move_alphabeta(board, depth=3, player='X')  # agent 1
        else:
            move = best_move_alphabeta(board, depth=2, player='O')  # agent 2 yếu hơn
        board = make_move(board, move, player)
        player = 'O' if player == 'X' else 'X'
    print_board(board)
    print("Winner:", get_winner(board))

play_heuristic_vs_heuristic()

## Tournament task [+1 to 5% bonus on your course grade; will be assigned separately]

Find another student and let your best agent play against the other student's best player. You are allowed to use any improvements you like as long as you code it yourself. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [10 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus point].

### Pure Monte Carlo Search

Implement Pure Monte Carlo Search (see [tic-tac-toe-example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_pure_monte_carlo_search.ipynb)) and investigate how this search performs on the test boards that you have used above. 

In [ ]:
# Your code/ answer goes here.
import random

def get_legal_moves(board):
    return [(r, c) for r in range(len(board)) for c in range(len(board[0])) if board[r][c] == '.']

def make_move(board, move, player):
    new_board = [row[:] for row in board]
    new_board[move[0]][move[1]] = player
    return new_board

def get_winner(board):
    # kiểm tra thắng theo hàng, cột, chéo
    for row in board:
        if row.count('X') == len(board): return 'X'
        if row.count('O') == len(board): return 'O'
    for c in range(len(board)):
        col = [board[r][c] for r in range(len(board))]
        if col.count('X') == len(board): return 'X'
        if col.count('O') == len(board): return 'O'
    diag1 = [board[i][i] for i in range(len(board))]
    diag2 = [board[i][len(board)-1-i] for i in range(len(board))]
    if diag1.count('X') == len(board) or diag2.count('X') == len(board): return 'X'
    if diag1.count('O') == len(board) or diag2.count('O') == len(board): return 'O'
    if all(board[r][c] != '.' for r in range(len(board)) for c in range(len(board))): return 'Draw'
    return None

def simulate_random_game(board, player):
    current = player
    while True:
        winner = get_winner(board)
        if winner is not None:
            return winner
        moves = get_legal_moves(board)
        if not moves: 
            return 'Draw'
        move = random.choice(moves)
        board = make_move(board, move, current)
        current = 'O' if current == 'X' else 'X'

def pure_monte_carlo(board, player, n_simulations=200):
    moves = get_legal_moves(board)
    best_move = None
    best_score = -1
    for move in moves:
        wins = 0
        for _ in range(n_simulations):
            result = simulate_random_game(make_move(board, move, player), player)
            if result == player:
                wins += 1
        score = wins / n_simulations
        if score > best_score:
            best_score = score
            best_move = move
    return best_move, best_score

# Ví dụ: chạy thử trên bàn 3x3
board = [['.','.','.'],
         ['.','.','.'],
         ['.','.','.']]
move, score = pure_monte_carlo(board, 'X', 300)
print("Best move:", move, "Win rate:", score)


### Best First Move

How would you determine what the best first move for a standard board ($5 \times 5$) is? You can use Pure Monte Carlo Search or any algorithms that you have implemented above.

In [ ]:
# Your code/ answer goes here.
# Xác định nước đi đầu tiên tốt nhất trên bàn 5x5 bằng Pure Monte Carlo
board = [['.']*5 for _ in range(5)]
move, score = pure_monte_carlo(board, 'X', 300)
print("Best first move for 5x5 board:", move, "Estimated win rate:", score)
